### NOTE: MedCAT uses an older version of scispacy. Please choose the kernel of "medcat" (in the top tab of "Kernel") to run MedCAT 

In [1]:
from medcat.cat import CAT
from medcat.utils.vocab import Vocab
from medcat.cdb import CDB


import pandas as pd
from medcat.meta_cat import MetaCAT
import numpy as np
import json
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2

### Load model

In [3]:
cdb = CDB()
# cdb.load_dict("/home/ubuntu/Tao/KER/medcat_models/0.2.6.2 - snomed_us_ext_names_umls_clean_primary_1M.dat")
cdb.load_dict("/home/ubuntu/Tao/KER/medcat_models/umls_base_wlink_clean_name_400k_mimic.dat")
vocab = Vocab()
vocab.load_dict(path='/home/ubuntu/Tao/KER/medcat_models/base_vocabulary.dat')

In [4]:
# gs = json.load(open("./paper_gs_used.json", 'r')) # Groups for some concepts
# chosen_cui = json.load(open("./paper_chosen_cui.json", 'r')) # Chosen based on F1 before

In [5]:
mc_negated = MetaCAT(save_dir="/home/ubuntu/Tao/KER/medcat_models/mc_negated/")
mc_negated.load()
mc_skip = MetaCAT(save_dir="/home/ubuntu/Tao/KER/medcat_models//mc_skip/")
mc_skip.load()

In [6]:
cat = CAT(cdb, vocab=vocab, meta_cats=[mc_negated, mc_skip])
cat.train = False
cat.spacy_cat.MIN_ACC = 0.35
cat.spacy_cat.MIN_CONCEPT_LENGTH = 2

# Set filters
# cat.spacy_cat.CUI_FILTER = set(json.load(open("./cui_filter.json")))
# cat.spacy_cat.TUI_FILTER = set(json.load(open("./tui_filter.json")))

### Load data

In [7]:
df = pd.read_csv("data/test_sample.csv")

In [8]:
df.head()

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text,gender,dob,dod,dod_hosp,dod_ssn,expire_flag,etl_date
0,674863,83982,147681.0,2145-04-02 00:00:00,2145-04-02 02:08:00,2145-04-02 04:31:23,Nursing,Nursing Progress Note,16436,NaN,"TITLE:\n This is a 55 y/o M, with H/O Hep C ...",M,2089-10-23 00:00:00,2145-05-04 00:00:00,2145-05-04 00:00:00,2145-05-04 00:00:00,1,2001-01-15 12:21:13
1,674864,50743,109381.0,2134-05-07 00:00:00,2134-05-07 02:19:00,2134-05-07 04:36:56,Nursing,Nursing Progress Note,20088,NaN,76 y/o M initially admitted to [**Hospital3 33...,M,2058-02-27 00:00:00,2134-06-06 00:00:00,2134-06-06 00:00:00,2134-06-06 00:00:00,1,2001-01-15 12:21:13
2,674865,96699,186382.0,2132-06-13 00:00:00,2132-06-13 04:36:00,2132-06-13 04:37:01,Respiratory,Respiratory Care Shift Note,15498,NaN,Demographics\n Day of intubation:\n Day of...,M,2079-01-01 00:00:00,2132-06-14 00:00:00,2132-06-14 00:00:00,NaN,1,2001-01-15 12:21:13
3,674925,96699,186382.0,2132-06-13 00:00:00,2132-06-13 06:37:00,2132-06-13 06:37:39,Nursing,Nursing Progress Note,16436,NaN,TITLE:\n Impaired Skin Integrity\n Assessm...,M,2079-01-01 00:00:00,2132-06-14 00:00:00,2132-06-14 00:00:00,NaN,1,2001-01-15 12:21:13
4,675045,75395,126239.0,2181-04-27 00:00:00,2181-04-27 08:01:00,2181-04-27 13:08:08,Nursing,Nursing Transfer Note,15659,NaN,"54yr man with hx metastatic renal cell ca, c/o...",M,2126-07-20 00:00:00,2181-05-22 00:00:00,2181-05-22 00:00:00,2181-05-22 00:00:00,1,2001-01-15 12:21:13


### Annotate text data

In [9]:
docs = {}

In [10]:
# The CSV must have the following three columns: id (row id), body_analysed (text of a document) and 
#client_idcode (patient id)

print(f"Len of df: {len(df)}")

batch_size = 1000 # Use 10k if you have a lot of documents
batch = []
cnt = 0
for name, text in df[['row_id', 'text']].values:
    text = str(text)
    cnt += 1
    if len(text) > 0 and name not in docs:
        # This will screwup positions of entites if doc>100k
        npart = int(np.ceil(len(text) / 100000))
        for i in range(npart):
            batch.append((name, text[i*100000:(i+1)*100000]))
        
        
    if len(batch) >= batch_size or (cnt == len(df) and len(batch) > 0):
        res = cat.multi_processing(batch, nproc=8, batch_size=batch_size // 100) # Use the correct number of processors

        for name, doc in res:
            if name not in docs:
                docs[name] = []
                
            docs[name].extend(doc['entities'])
#             print(name)
        batch = []

Len of df: 1000


Process Process-6:
Process Process-4:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/medcat/cat.py", line 818, in _mp_cons
    doc = json.loads(self.get_json(text, cat_filter, only_cui, skip_info))
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/medcat/cat.py", line 741, in get_json
    ents = self.get_entities(text, cat_filter, only_cui, skip_info=skip_info)
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/medcat/cat.py", line 695, in get_entities
    doc = self(text)
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/medcat/cat.py", line 94, in __call__
    return self.nlp(text)
Pro

KeyboardInterrupt: 

  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process Process-3:
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/medcat/cat.py", line 741, in get_json
    ents = self.get_entities(text, cat_filter, only_cui, skip_info=skip_info)
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/medcat/cat.py", line 818, in _mp_cons
    doc = json.loads(self.get_json(text, cat_filter, only_cui, skip_info))
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/medcat/cat.py", line 695, in get_entities
    doc = self(text)
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/medcat/cat.py", line 741, in get_json
    ents = self.get_entities(text, cat_filter, only_cui, skip_info=skip_info)
Traceback (most recent call last):
Process Process-2:
  File "/home/ubuntu/anaconda3/envs/medcat

  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/medcat/utils/models.py", line 53, in forward
    x, hidden = self.rnn(x)
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/spacy/language.py", line 431, in __call__
    doc = self.make_doc(text)
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/ubuntu/anaconda3/envs/medcat37/lib/python3.7/site-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/ubuntu/anaconda3/envs/m

In [7]:
# # Combine for groups
# for doc in docs:
#     for i in range(len(docs[doc])):
#         ann = docs[doc][i]

#         for key, value in gs.items():
#             if ann['cui'] in value:
#                 ann['cui'] = key

In [53]:
# Map from document id to patient id
doc2id = {}
for doc, ptid in df[['row_id', 'subject_id']].values:
    doc2id[doc] = ptid

### MedCAT can annoate negated and irrelevant mentions of a concept

In [54]:
# Find annotations per patient/document and counts for cui
pt_added = {}
doc2cuic = {}
for doc in docs.keys():
    doc2cuic[doc] = []
    id = doc2id[doc]
    for ann in docs[doc]:
        negation = False
        skip = False
        cui = ann['cui']
        
#         print(ann['meta_anns']['Negated']['value'])
#         for meta_ann in ann['meta_anns']:
        if ann['meta_anns']['Negated']['value'] == 'No':
            negation = True
        if ann['meta_anns']['Skip']['value'] == 'No':
            skip = True

        if negation and skip:
#             print(negation, skip)
            if id not in pt_added:
                pt_added[id] = {cui: 1}
            elif cui not in pt_added[id]:
                pt_added[id][cui] = 1
            else:
                pt_added[id][cui] += 1
                
            doc2cuic[doc].append(ann)

In [56]:
# A map for each patent from pt_id -> [<cuis>]
pt_added_chosen = {} # Only patients > min_count and for the chosen cuis
threshold = 2
for pt in pt_added:
    pt_added_chosen[str(pt)] = []
    for cui in pt_added[pt]:
        if pt_added[pt][cui] >= threshold:
            pt_added_chosen[str(pt)].append(cui)
# These are the final annotations that should be used
# print(pt_added_chosen)
json.dump(pt_added_chosen, open("data/pt2cuis.json", 'w'))

In [57]:
# Save also the names for the CUIs
cui2name = {}
for pt in pt_added:
    for cui in pt_added[pt]:
        cui2name[cui] = cdb.cui2pretty_name[cui]
json.dump(cui2name, open("data/cui2name.json", 'w'))

In [50]:
cui2name

{'S-225757006': 'Every four to six hours (qualifier value)',
 'S-272379006': 'Event (event)',
 'S-30294006': 'With trend (attribute)',
 'S-263490005': 'Status (attribute)',
 'S-304253006': 'Not for resuscitation (finding)',
 'S-35105006': 'Increased (qualifier value)',
 'S-13543005': 'Pressure (finding)',
 'S-738985004': 'Pulmonary (intended site)',
 'S-42796001': 'End-stage (qualifier value)',
 'S-50360004': 'Reading (observable entity)',
 'S-734005006': 'Telephone number (property) (qualifier value)',
 'S-412383006': 'Dopamine (substance)',
 'S-37328000': 'Weaning, function (observable entity)',
 'S-255605001': 'Minimal (qualifier value)',
 'S-250230009': 'Haematocrit (procedure)',
 'S-419582001': 'Give - dosing instruction imperative (qualifier value)',
 'S-246514001': 'Units (attribute)',
 'S-350177006': 'Oral form sulfamethoxazole + trimethoprim (product)',
 'S-372735009': 'Vancomycin (substance)',
 'S-703743007': 'Piperacillin and tazobactam (substance)',
 'S-36576007': 'Infusion

## Note: the Snomed concept IDs output from MedCAT are slightly different from those in the original Snomed. A prefix of "S-" is added to MedCAT outputs.  